## Custom Module for Twitter OAuth1

### 3-legged OAuth

This section supplements Twitter’s 3-legged OAuth flow documentation.

To authenticate as a user other than your developer account, you’ll need to obtain their access tokens through the 3-legged OAuth flow.

First, you’ll need to turn on OAuth 1.0 under the User authentication settings section of your app’s Settings tab under the Twitter Developer Portal Projects & Apps page. To do this, you’ll need to provide a Callback / Redirect URI / URL.

Then, you’ll need the app’s API / `consumer key` and `secret` that can be found through your app’s Keys and Tokens tab under the Twitter Developer Portal Projects & Apps page.

In [3]:
import tweepy

In [4]:

class TwitterOAuth1:
    def __init__(self, consumer_key, consumer_secret, callback_url='oob', oauth_token=None, oauth_token_secret=None):
        self.consumer_key = consumer_key # consumer_key from the Twitter Devloper portal
        self.consumer_secret = consumer_secret # consumer_key from the Twitter Devloper portal
        self.callback_url = callback_url # Priority add to add `CallBack URLs` at Twitter Devloper portal
        self.oauth_token = oauth_token # Token of OAuth1UserHandler.get_authorization_url()
        self.oauth_token_secret = oauth_token_secret # Token Secret of OAuth1UserHandler.get_authorization_url()
        self.access_token = None  # Access_Token is the KEY for twitter account access of Authorized Accounts
        self.access_token_secret = None # Access_Token_SECRET is the SECRET_KEY for twitter account access of Authorized Accounts
        
    def oauth1_user_handler(self):
        
        # inititate OAuth1UserHandler
        return tweepy.OAuth1UserHandler(
            CONSUMER_KEY, CONSUMER_SECRET,
            callback=CALLBACK
        )
    def get_authorization_url_for_client(self):
        oauth1_user_handler = self.oauth1_user_handler()
        # Generate the Authorization URL of Twitter account for Code level access
        authorization_url = oauth1_user_handler.get_authorization_url()
        self.oauth_token = oauth1_user_handler.request_token['oauth_token']
        self.oauth_token_secret = oauth1_user_handler.request_token['oauth_token_secret']
        return authorization_url
    
    def signature_verify_with_verifier(self, oauth_verifier):
        
        # inititate OAuth1UserHandler
        oauth1_user_handler = self.oauth1_user_handler()
        oauth1_user_handler.request_token = {
            "oauth_token": self.oauth_token,
            "oauth_token_secret": self.oauth_token_secret
        }
        self.access_token, self.access_token_secret = (
            oauth1_user_handler.get_access_token(oauth_verifier)
        )
        return self.access_token, self.access_token_secret


## Example

### Step 1: Authorization Request Code

In [9]:
CONSUMER_KEY = "<consumer key from Twitter App>"
CONSUMER_SECRET = "<consumer SECRET from Twitter App>"
CALLBACK = "https://www.callback.url"

In [11]:
twitter_oauth1 = TwitterOAuth1(
    consumer_key=CONSUMER_KEY,
    consumer_secret=CONSUMER_SECRET,
    callback_url=CALLBACK
)
authorization_url = twitter_oauth1.get_authorization_url_for_client()

# To be save in DB
oauth_token = twitter_oauth1.oauth_token
oauth_token_secret = twitter_oauth1.oauth_token_secret

print(f"authorization_url: {authorization_url}")
print(f"oauth_token: {oauth_token}")
print(f"oauth_token_secret: {oauth_token_secret}")

TweepyException: Token request failed with code 401, response was '{"errors":[{"code":32,"message":"Could not authenticate you."}]}'.

#### Callback URL

`http://127.0.0.1:8000/twitter/callback/?oauth_token=FEjcSgAAAAABlnTDAAABhiso1uM&oauth_verifier=uaskiMkVDIVDaLZarPyX8HmfHzi1TNvv`

### Callback Code

In [ ]:
oauth_token = "<oauth_token value which save from above function>" # From DB
oauth_token_secret = "<oauth_token_secret value which save from above function>" # From DB
oauth_verifier = "<oauth_verifier from the callback url>" # From URL

new_twitter_oauth1 = TwitterOAuth1(
    # From Env
    consumer_key=CONSUMER_KEY,
    consumer_secret=CONSUMER_SECRET,
    callback_url=CALLBACK,
    # From DB
    oauth_token=oauth_token,
    oauth_token_secret=oauth_token_secret
)

access_key, access_secret = new_twitter_oauth1.signature_verify_with_verifier(oauth_verifier)
print(f"access_key: {access_key}")
print(f"access_secret: {access_secret}")